<a href="https://colab.research.google.com/github/arshad-huh/100-Days-Challenge/blob/main/22_da_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

User Engagement Analysis For RESTAURANT SUCCESS

About YELP

Yelp is a web and mobile platform that functions as a crowd-sourced local
business review site. Users can submit reviews, photos, and tips about
businesses, while also browsing information and ratings left by others.

AGENDA
• Problem Statement
• Research Objectives
• Hypothesis
• Data Overview
• Analysis and Findings
• Recommendations

Problem Statement

In a competitive market like the restaurant industry, understanding the
factors that influence business success is crucial for stakeholders.
Utilizing the Yelp dataset, this project aims to investigate the relationship
between user engagement (reviews, tips, and check-ins) and business
success metrics (review count, ratings) for restaurants.

Rsearch Objectives

Quantity the correlation
between user engagement
(reviews, tips, check-ins) and
review count/ average star
rating.

Analyze the impact of
sentiment on review count and
average star rating.

Time Trends in User
Engagement

Hypothesis

• Higher levels of user engagement (more reviews, tips, and check-ins)
correlate with higher review counts and ratings for restaurants.

• Positive sentiment expressed in reviews and tips contributes to higher
overall ratings and review counts for restaurants.

• Consistent engagement over time is positively associated with
sustained business success for restaurants.

Data Overview

• This dataset is a subset of Yelp and has information about busir
across 8 metropolitan areas in the USA and Canada.

• The original data is shared by Yelp as JSON files.

• The five JSON files are business, review, user, tip and checkin.

• The JSON files are stored in the database for easy retrieval of data.

Analysis and Findings


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'yelp-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10100%2F3316532%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240530%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240530T074316Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D72098f4777054f9473f9289a3b2d4a208a6a11b17135c8aa499b70c6454ea8ba93e7f38730c8063920687f462d9dbdd9cb1054d69afa8a5ecee37ad47c97ac5acd7890a49f34187aae550e6656ff651d9be103e9461cc42491f02a2666b8d5a769c8a4bf1d40c1645b1ce3eb4989dfc7a9128acd2256a432c951023af70bd8df5800864c9bd33a696d49bdfad6d84e400dbbcf3d21eb75755310a8f2cf666a73c36368eaf27ed0b283918e8420de768a2fa5efc73453deab2663443e3151920947fa0342a1ee79391d8905d11bd8e6998c09c68966a503ae406096804b7ee1c7ac9a22d669d0b83e69e48f1a69a28d8151336f35e53c7b260ca7e73e8c2bfa26'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 4374983563 bytes downloaded
Downloaded and uncompressed: yelp-dataset
Data source import complete.


In [ ]:
!pip install sqlalchemy

In [ ]:
import pandas as pd
import json
from sqlalchemy import create_engine

with open('/input/yelp-dataset/yelp_academic_dataset_business.json', 'r') as f:
  business_data = [json.loads(line) for line in f]
business_df = pd.DataFrame(business_data)

with open('/input/yelp-dataset/yelp_academic_dataset_checkin.json', 'r') as f:
  checkin_data = [json.loads(line) for line in f]
checkin_df = pd.DataFrame(checkin_data)

with open('/input/yelp-dataset/yelp_academic_dataset_review.json', 'r') as f:
  review_data = [json.loads(line) for line in f]
review_df = pd.DataFrame(review_data)

with open('/input/yelp-dataset/yelp_academic_dataset_tip.json', 'r') as f:
  tip_data = [json. loads(line) for line in f]
tip_df = pd.DataFrame(tip_data)

with open('/input/yelp-dataset/yelp_academic_dataset_user.json', 'r') as f:
  user_data = [json.loads(line) for line in f]
user_df = pd.DataFrame(user_data)

In [ ]:
print(business_df.shape)
print(checkin_df.shape)
print(review_df.shape)
print(tip_df.shape)
print(user_df.shape)

In [ ]:
business_df.drop(['attributes', 'hours'], axis=1, inplace=True)

In [ ]:
engine = create_engine('sqlite:///yelp.db')

def load_dataframe(df, table_name, engine):
  df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Load each DataFrame into a separate table
load_dataframe(business_df, 'business', engine)
load_dataframe(review_df, 'review', engine)
load_dataframe(user_df, 'user', engine)
load_dataframe(tip_df, 'tip', engine)
load_dataframe(checkin_df, 'checkin', engine)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
import sqlite3
import folium
from geopy.geocoders import Nominatin
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

Database Connection

In [ ]:
conn = sqlite3.connect('yelp.db')